# Challenge Set 1

Topic:  Explore MTA turnstile data

Date:   01/25/2016

Name:   Daniel Yawitz


In [ ]:
import os, csv, re
f = 'turnstile_160109.txt'
#f = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_160109.txt"

In [ ]:
#Read in the data:
def read_data(file):
    data = open(file)
    d = []
    csv_data = csv.reader(data)
    for row in csv_data:
        #strip extra spaces from 'EXITS'
        row[10] = row[10].rstrip()
        d.append(row)
    return d
    data.close()

d = read_data(f)
#pop out the headers
d.pop(0)
#take a small sample of the data
d = d[:1000]

'''format of 'd' is a list of lists, for all 11 variables'''

# Challenge 1
Make a dictionary so that each turnstile is a key and its 4 hour measurements are values

In [ ]:
def turn_dict(d):
    '''turns the raw data into a dict of the form {turnstile:[values]}
    
    One example value:
    {('A006', 'R079', '00-03-02', '5 AV/59 ST'):
    [['NQR',
   'BMT',
   '01/02/2016',
   '01:00:00',
   'REGULAR',
   '0013206367',
   '0013861091'],
   
  ['NQR',
   'BMT',
   '01/02/2016',
   '05:00:00',
   'REGULAR',
   '0013206371',
   '0013861092']}
    
    '''
    #create empty dict
    mta_dict = {}
    #pop out the headers
#     if d[0] == ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME',
#                        'DIVISION', 'DATE', 'TIME', 'DESC', 'ENTRIES',
#                        'EXITS']:
#         d.pop(0)
    
    
    for row in d:
        
        #key is the first 4 columns that ID the turnstile
        key = tuple(row[0:4])
        #values are the rest of the data
        value = row[4:11]
        
        #This line does the same thing as the if/else statement below
        mta_dict.setdefault(key, []).append(value) 
        
        #if key in mta_dict:
        #    mta_dict[key].append(value)
        #else:
        #    mta_dict[key] = [value]
            
       
        
    return mta_dict

turnstile_dict = turn_dict(d)

turnstile_dict is a solution to challenge 1

# Challenge 2
Create a new dictionary. The key is the same, but the values are a list of datetime objects and entry values

In [ ]:
import datetime
import dateutil.parser

def get_datetime_cumulative(d):
    ''' takes in the raw data and exports a dict of the form {turnstile_key: [[datetime_object, counts]]}
     {('A006', 'R079', '00-03-02', '5 AV/59 ST'):
        [[datetime.datetime(2016, 1, 2, 1, 0), '0013206367'],
         [datetime.datetime(2016, 1, 2, 5, 0), '0013206371']]}
    
    '''
    datetime_cumulative = {}
    
    #pop out the headers
    if d[0] == ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME',
                       'DIVISION', 'DATE', 'TIME', 'DESC', 'ENTRIES',
                       'EXITS']:
        d.pop(0)
    
    for row in d:
        #get datetime object. Row 6 and 7 of 'd' are '2013-12-02' and '15:00:00'
        dt_str = row[6] + ' ' + row[7]
        dt = dateutil.parser.parse(dt_str)
        #row 9 is turnstile entries
        entries = row[9]

        key = tuple(row[0:4])
        values = [dt, entries]
        
        #This line does the same thing as the if/else statement below
        datetime_cumulative.setdefault(key, []).append(values)

        #if key in datetime_cumulative:
        #    datetime_cumulative[key].append(values)
        #else:
        #    datetime_cumulative[key] = [values]
        
        
    return datetime_cumulative

datetime_cumulative = get_datetime_cumulative(d)

datetime_cumulative is a solution to challenge 2

# Challenge 3
Use the same keys, but the values should be the total number of passengers for each day per turnstile

In [ ]:
def count_day(my_list):
    #Takes in a list of lists (the values for one turnstile)
    #datetime_cumulative[key]
    #Output is a list of lists that gets the counts for one turnstile's day
    
    ''' [[datetime.datetime(2016, 1, 2, 1, 0), '0013206367'],
         [datetime.datetime(2016, 1, 2, 5, 0), '0013206371']]
         
         -->
         
         [[dateime.datetime(2016, 1,2), 2]]
        
         
         '''
    
    ## Get unique values of dates
    dates = [row[0].date() for row in my_list]
    #make it a set to eliminate duplicates
    dates = set(dates)
    dates = sorted(list(dates))

    result = list()

    for date in dates:
        
        #get tomorrows date. Add one 'day' to today
        tomorrows_date = date + datetime.timedelta(days=1)
        
        #for every entry with today's date, add the count to a list
        todays_entries = [int(x[1]) for x in my_list if x[0].date() == date]
        
         todays_entries = [int(x[2]) for x in rows if x[0].date() == date]
        #
        if tomorrows_date in dates:
            #if tomorrow's date is in dates, compare the start of tomorrow to the end of today
            tomorrows_entries = [int(x[1]) for x in my_list if x[0].date() == tomorrows_date]
            delta = min(tomorrows_entries)- min(todays_entries)
            
        else:
            #otherwise, use the change in today
            delta = max(todays_entries) - min(todays_entries)
        
        result.append((date, delta))
    return result

In [ ]:
turnstile_counts = {key:count_day(datetime_cumulative[key]) for key in datetime_cumulative}
''' one {key:value} pair in turnstile_counts looks like: 

{('A002', 'R051', '02-03-06', '59 ST'),
 [(datetime.date(2016, 1, 4), 2278),
  (datetime.date(2016, 1, 5), 2326),
  (datetime.date(2016, 1, 8), 2325),
  (datetime.date(2016, 1, 2), 1159),
  (datetime.date(2016, 1, 3), 915),
  (datetime.date(2016, 1, 6), 2474),
  (datetime.date(2016, 1, 7), 2553)]}

'''

# Challenge 4

Plot the daily time series for a turnstile.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def plot_turnstile(dictionary, turnstile_key):
    turnstile_data = sorted(dictionary[turnstile_key])

    dates = [x[0] for x in turnstile_data]
    counts = [x[1] for x in turnstile_data]

    plt.figure(figsize=(10,3))
    plt.plot(dates,counts)

In [ ]:
key = ('A035', 'R170', '00-00-04', '14 ST-UNION SQ')
plot_turnstile(turnstile_counts, key)

# Challenge 5 & 6
This is just the code and functions. Scroll down to see the solutions. 

In [ ]:
def tcounts_to_dailydata(tcounts):
    ''' dict --> list of lists

        restructures turnstile_counts into a list of lists that looks like the original data:

        ['A002', 'R051', '02-03-06', '59 ST', datetime.date(2016, 1, 4), 2278]
    '''
    dailydata = []

    #get a list of the keys, so you can iterate over something 
    turnstiles = tcounts.keys()

    for t in turnstiles:
        for datecounts in tcounts[t]:
            dailydata += [[t[0], t[1], t[2], t[3], datecounts[0], datecounts[1]]]

    return dailydata

dailydata = tcounts_to_dailydata(turnstile_counts)

In [ ]:
def dailydata_to_ac_stationdict(dailydata):
    ''' dict --> dict

    restructures turnstile_counts so that the key is station info (including AC), and the value includes the turnstile ID:

    {('A007', 'R079', '5 AV/59 ST'): [['01-05-01', datetime.date(2016, 1, 4), 0],
      ['01-05-01', datetime.date(2016, 1, 5), 0],
      ['01-05-01', datetime.date(2016, 1, 8), 0],
      ['01-05-01', datetime.date(2016, 1, 2), 0],
      ['01-05-01', datetime.date(2016, 1, 3), 0],
      ['01-05-01', datetime.date(2016, 1, 6), 0],
      ['01-06-01', datetime.date(2016, 1, 4), 1413],
      ['01-06-01', datetime.date(2016, 1, 5), 1204],
      ['01-06-01', datetime.date(2016, 1, 2), 1135],
      ['01-06-01', datetime.date(2016, 1, 3), 847],
      ['01-06-01', datetime.date(2016, 1, 6), 1448],
      ['01-06-01', datetime.date(2016, 1, 7), 172]] }

    '''
    stationdict = {}
    
    for i in range(len(dailydata)-1):
        ca = dailydata[i][0]
        unit = dailydata[i][1]
        scp = dailydata[i][2]
        station = dailydata[i][3]
        day = dailydata[i][4]
        count = dailydata[i][5]

        key = (ca, unit, station)
        value = [scp, day, count]

        stationdict.setdefault(key, []).append(value) 

    # stationdict = {tuple(list(turnstile[0:2])+[turnstile[3]]): 
    #                                  [[turnstile[2],
    #                                     datecounts[i][0],
    #                                     datecounts[i][1]] 
    #                                 for i in range(len(datecounts)-1)]
    #                 for turnstile, datecounts in tcounts.iteritems()}

    return stationdict

ac_stationdict = dailydata_to_ac_stationdict(dailydata)

In [ ]:
def dailydata_to_stationdict(dailydata):
    ''' dict --> dict

    restructures turnstile_counts so that the key is station info, and the value includes the turnstile ID:

    {('5 AV/59 ST'): [['01-05-01', datetime.date(2016, 1, 4), 0],
      ['01-05-01', datetime.date(2016, 1, 5), 0],
      ['01-05-01', datetime.date(2016, 1, 8), 0],
      ['01-05-01', datetime.date(2016, 1, 2), 0],
      ['01-05-01', datetime.date(2016, 1, 3), 0],
      ['01-05-01', datetime.date(2016, 1, 6), 0],
      ['01-06-01', datetime.date(2016, 1, 4), 1413],
      ['01-06-01', datetime.date(2016, 1, 5), 1204],
      ['01-06-01', datetime.date(2016, 1, 2), 1135],
      ['01-06-01', datetime.date(2016, 1, 3), 847],
      ['01-06-01', datetime.date(2016, 1, 6), 1448],
      ['01-06-01', datetime.date(2016, 1, 7), 172]] }

    '''
    stationdict = {}
    
    for i in range(len(dailydata)-1):
        ca = dailydata[i][0]
        unit = dailydata[i][1]
        scp = dailydata[i][2]
        station = dailydata[i][3]
        day = dailydata[i][4]
        count = dailydata[i][5]

        key = (station)
        value = [scp, day, count]

        stationdict.setdefault(key, []).append(value) 

    # stationdict = {tuple(list(turnstile[0:2])+[turnstile[3]]): 
    #                                  [[turnstile[2],
    #                                     datecounts[i][0],
    #                                     datecounts[i][1]] 
    #                                 for i in range(len(datecounts)-1)]
    #                 for turnstile, datecounts in tcounts.iteritems()}

    return stationdict

stationdict = dailydata_to_stationdict(dailydata)

In [ ]:
def get_stationcounts(stationdict):
    ''' Takes the stationdict and adds up the daily totals for all those turnstiles.
    
    {('A047', 'R087', 'CITY HALL'):
    [[datetime.date(2016, 1, 4), 1020],
    [datetime.date(2016, 1, 5), 815],
    [datetime.date(2016, 1, 6), 1030]]}
    
    '''
    stations = stationdict.keys()
    
    stationcounts = {}
    
    for key in stations:
        rows = stationdict[key]
        
        dates = [row[1] for row in rows]
        dates = set(dates)
        dates = sorted(list(dates))
        
        day_sums = list()

        for date in dates:
            todays_entries = [int(x[2]) for x in rows if x[1] == date]
            daily_sum = sum(todays_entries)
            day_sums.append([date, daily_sum])
        stationcounts.setdefault(key,[]).append(day_sums)
    
    return stationcounts

stationcounts = get_stationcounts(stationdict)

# Challenge 5
-combine the numbers together -- for each ControlArea/UNIT/STATION combo, for each day, add the counts from each turnstile belonging to that combo
-make this a new dict

In [ ]:
challenge5 = get_stationcounts(ac_stationdict)

print challenge5.items()[0]

# Challenge 6
Create a dictionary for stations and their daily entries

In [ ]:
stationcounts = get_stationcounts(stationdict)
print stationcounts.items()[1:2]